### Tweet Data Collection
This notebook does the following:
- Collects the twitter data from the bounding box of geo-cordinates specified below.
- Outputs the twitter data in the json output file specified below.

In [ ]:
outputJsonFile = 'datanyctest.json'
locationBoundingBox = [-74,40,-73,41]

In [ ]:
import tweepy
from tweepy import OAuthHandler
from tweepy import StreamListener
from tweepy import Stream
import json
import os

In [ ]:
consumer_key = ''
consumer_secret = ''
access_token = ''
access_secret = ''

In [ ]:
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth)

In [ ]:
@classmethod
def parse(cls, api, raw):
    status = cls.first_parse(api, raw)
    setattr(status, 'json', json.dumps(raw))
    return status
 
# Status() is the data model for a tweet
tweepy.models.Status.first_parse = tweepy.models.Status.parse
tweepy.models.Status.parse = parse

In [ ]:
class MyListener(StreamListener):
 
    def on_data(self, data):
        storagefile = outputJsonFile
        try:
            if os.path.isfile(storagefile): 
                print('1')
                with open(storagefile) as json_file:  
                    oldData = json.load(json_file)
                data = oldData + [json.loads(data)]
                with open(storagefile, 'w') as outfile:
                    json.dump(data, outfile)
                    return True
            else:
                print('2')
                data = [json.loads(data)]
                with open(storagefile, 'w') as outfile:
                    json.dump(data, outfile)
                    return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
            return True
 
    def on_error(self, status):
        print(status)
        return True

In [ ]:
while(True):
    try:
        twitter_stream = Stream(auth, MyListener())
        twitter_stream.filter(locations=locationBoundingBox)
    except Exception as e:
        print("Error: %s" % str(e))